In [2]:
import pandas as pd
import numpy as np

In [3]:
#Read csv file containing ONS defined coastal towns - https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/1005216/cmo-annual_report-2021-health-in-coastal-communities-accessible.pdf - appendix 2 - p233
df_ONS_coastal = pd.read_csv('ONS_coastal_BUA_BUASU_(2021).csv')

#Read csv file containing ONS MSOA lookup file - https://geoportal.statistics.gov.uk/datasets/lower-layer-super-output-area-2011-to-built-up-area-sub-division-to-built-up-area-to-local-authority-district-to-region-december-2011-lookup-in-england-and-wales/explore
df_LSOA_to_BUA_BUASD = pd.read_csv('LSOA_(2011)_to_BUASD_to_BUA_to_LAD_to_Region_(December_2011)_Lookup_in_England_and_Wales.csv', usecols=['LSOA11CD','BUASD11CD','BUA11CD'])

#Read csv file containing ONS LSOA to IMD lookup file - https://geoportal.statistics.gov.uk/datasets/index-of-multiple-deprivation-december-2019-lookup-in-england/explore
df_LSOA_to_IMD = pd.read_csv('Index_of_Multiple_Deprivation_(December_2019)_Lookup_in_England.csv', usecols=['LSOA11CD','IMD19'])

In [4]:
df_BUASD = pd.merge(df_LSOA_to_BUA_BUASD, df_ONS_coastal, how='inner', left_on = 'BUASD11CD', right_on = 'Area_Code')
df_BUA = pd.merge(df_LSOA_to_BUA_BUASD, df_ONS_coastal, how='inner', left_on = 'BUA11CD', right_on = 'Area_Code')
df_coastal_lookup = pd.concat([df_BUASD, df_BUA])

df_coastal_lookup = pd.merge(df_coastal_lookup, df_LSOA_to_IMD, how='inner', left_on = 'LSOA11CD', right_on = 'LSOA11CD')

In [47]:
conditions = [
    (df_coastal_lookup['IMD19'] >= 0) & (df_coastal_lookup['IMD19'] < 32844*1/5),
    (df_coastal_lookup['IMD19'] >= 32844*1/5) & (df_coastal_lookup['IMD19'] < 32844*2/5),
    (df_coastal_lookup['IMD19'] >= 32844*2/5) & (df_coastal_lookup['IMD19'] < 32844*3/5),
    (df_coastal_lookup['IMD19'] >= 32844*3/5) & (df_coastal_lookup['IMD19'] < 32844*4/5),
    (df_coastal_lookup['IMD19'] >= 32844*4/5) & (df_coastal_lookup['IMD19'] < 32844),
    ]

# create a list of the values we want to assign for each condition
values = [1, 2, 3, 4, 5]

# create a new column and use np.select to assign values to it using our lists as arguments
df_coastal_lookup['IMDQ'] = np.select(conditions, values)

In [56]:
df_coastal_lookup.head()
areas=df_coastal_lookup['Area_Name'].unique()

area_column=[]
minimd_column=[]
maximd_column=[]
meanimd_column=[]
medianimd_column=[]

for area in areas:
    mask = df_coastal_lookup[df_coastal_lookup['Area_Name'] == area]
    area_column.append(area)
    minimd_column.append(mask['IMDQ'].min())
    maximd_column.append(mask['IMDQ'].max())
    meanimd_column.append(mask['IMDQ'].mean())
    medianimd_column.append(mask['IMDQ'].median())


d = {'Area_Name': area_column,'IMDmin':minimd_column, 'IMDmax': maximd_column, 'IMDmean': meanimd_column, 'IMDmedian': medianimd_column}
df_min_max_imd = pd.DataFrame(d)

In [58]:
df_min_max_imd.to_csv('Coastal_min_max_IMD_output.csv', index=False)  

df_min_max_imd

,Area_Name,IMDmin,IMDmax,IMDmean,IMDmedian
0,Liverpool BUASD,1,5,1.739496,1.0
1,Crosby BUASD,1,5,2.685714,3.0
2,Bootle BUASD,1,4,1.282051,1.0
3,Birkenhead BUASD,1,5,2.076923,1.0
4,Hoylake BUASD,2,5,3.857143,4.0
...,...,...,...,...,...
147,Newbiggin-by-the-Sea BUA,1,2,1.250000,1.0
148,Minehead BUA,2,3,2.714286,3.0
149,Lowestoft BUA,1,5,2.304348,2.0
150,Selsey BUA,2,4,3.142857,3.0


In [12]:
df_coastal_lookup.to_csv('Coastal_IMD_output.csv', index=False)  

df_coastal_lookup

,LSOA11CD,BUASD11CD,BUA11CD,Area_Code,Area_Name,Region/Country,Area_classification,IMD19,IMDQ
0,E01006514,E35001470,E34004801,E35001470,Liverpool BUASD,North West,Coastal city,3299,1
1,E01006412,E35001470,E34004801,E35001470,Liverpool BUASD,North West,Coastal city,264,1
2,E01006413,E35001470,E34004801,E35001470,Liverpool BUASD,North West,Coastal city,97,1
3,E01006515,E35001470,E34004801,E35001470,Liverpool BUASD,North West,Coastal city,1875,1
4,E01006465,E35001470,E34004801,E35001470,Liverpool BUASD,North West,Coastal city,16357,3
...,...,...,...,...,...,...,...,...,...
4110,E01031381,NaN,E34004154,E34004154,Bognor Regis BUA,South East,Larger seaside town,22226,4
4111,E01031382,NaN,E34004154,E34004154,Bognor Regis BUA,South East,Larger seaside town,20579,4
4112,E01031383,NaN,E34004154,E34004154,Bognor Regis BUA,South East,Larger seaside town,29850,5
4113,E01031384,NaN,E34004154,E34004154,Bognor Regis BUA,South East,Larger seaside town,27897,5
